**Exploring Data**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
print(check_output(["ls", "../input"]).decode("utf8"))

images_sample
sample_submission.csv
test.json
train.json



In [2]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.head()

,listing_id,high,medium,low
0,7142618,0.077788,0.227529,0.694683
1,7210040,0.077788,0.227529,0.694683
2,7174566,0.077788,0.227529,0.694683
3,7191391,0.077788,0.227529,0.694683
4,7171695,0.077788,0.227529,0.694683


In [3]:
df = pd.read_json('../input/train.json')
df.tail()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
9999,1.0,2,297160caf8a88c10718b492d4672c6b4,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,"[Elevator, Laundry in Building, Laundry in Uni...",medium,40.7426,7098690,-73.9790,2a1ee03b449700c3a15dd8c9a505c525,[https://photos.renthop.com/2/7098690_18396d32...,3200,230 E 30 St
99991,1.0,1,9e871ccfea7b07686c101548ca693af6,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",low,40.7102,6822449,-74.0163,ef20d40b528500037e37b167787ac089,[https://photos.renthop.com/2/6822449_b429587b...,3950,225 Rector Place
99992,1.0,1,7967a1280bf3f7644500fc79d2696b0e,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,"[Doorman, Elevator, Pre-War, Dogs Allowed, Cat...",low,40.7601,6881461,-73.9900,d889a9552265fe359e7d36d172084fcb,[https://photos.renthop.com/2/6881461_20a86530...,2595,341 West 45th Street
99993,1.0,0,ad67f6181a49bde19218929b401b31b7,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,"[Doorman, Elevator, Pre-War, Dogs Allowed, Cat...",low,40.7066,6841891,-74.0101,9fd3af5b2d23951e028059e8940a55d7,[https://photos.renthop.com/2/6841891_124c9c44...,3350,37 Wall Street
99994,1.0,2,5173052db6efc0caaa4d817112a70f32,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,[Hardwood Floors],low,40.8699,6858245,-73.9172,d7f57128272bfd82e33a61999b5f4c42,[https://photos.renthop.com/2/6858245_c4380bde...,2200,30 Park Terrace East


In [4]:
print('Unique: ', len(set(df['manager_id'])))
print('Total: ', len(df['manager_id']))
print(len(df['manager_id'])/len(set(df['manager_id'])))

Unique:  3481
Total:  49352
14.177535191037059


In [5]:
df['address'] = df['display_address'].astype('category').cat.codes
df['street_address'] = df['street_address'].astype('category').cat.codes
df['building_id'] = df['building_id'].astype('category').cat.codes
df['manager_id'] = df['manager_id'].astype('category').cat.codes
df['num_features'] = df['features'].apply(len)
df['created'] = pd.to_datetime(df['created'])
df['created_year'] = df['created'].dt.year.astype('category').cat.codes
df['created_month'] = df['created'].dt.month.astype('category').cat.codes
df['len_description'] = df['description'].apply(lambda x: len(x.split(' ')))
df['num_pics'] = df['photos'].apply(len)

In [6]:
new_feat = ['price','address','manager_id','building_id',
            'num_features','created_year','created_month',
            'len_description','latitude','longitude','num_pics']

#new_feat = ['price','latitude','longitude','num_pics',
 #           'num_features','created_year','created_month','len_description']
X = df[new_feat].fillna(0)
y = df['interest_level'].astype('category').cat.codes
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=9)
X.tail()

,price,address,manager_id,building_id,num_features,created_year,created_month,len_description,latitude,longitude,num_pics
9999,3200,4841,597,1219,6,0,2,115,40.7426,-73.9790,4
99991,3950,6887,3251,4693,8,0,0,191,40.7102,-74.0163,8
99992,2595,8423,2958,3570,5,0,0,84,40.7601,-73.9900,3
99993,3350,8126,2171,5150,5,0,0,99,40.7066,-74.0101,9
99994,2200,6789,2952,2351,1,0,0,133,40.8699,-73.9172,7


In [7]:
clf1 = GradientBoostingClassifier(n_estimators=200, max_depth=9)
clf2 = AdaBoostClassifier(n_estimators=200)
clf3 = RandomForestClassifier(n_estimators=300)

estimators = [('gb', clf1), ('ab', clf2), ('rf', clf3)]
vclf = VotingClassifier(estimators=estimators, voting='soft', n_jobs= -1)

vclf.fit(X_train, y_train)
y_val_pred = vclf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.69895162380759213

In [8]:
X_train = df[new_feat].fillna(0)
y_train = df['interest_level']
vclf.fit(X_train, y_train)


df2 = pd.read_json('../input/test.json')
df2['address'] = df2['display_address'].astype('category').cat.codes
df2['street_address'] = df2['street_address'].astype('category').cat.codes
df2['building_id'] = df2['building_id'].astype('category').cat.codes
df2['manager_id'] = df2['manager_id'].astype('category').cat.codes
df2['num_features'] = df2['features'].apply(len)
df2['created'] = pd.to_datetime(df2['created'])
df2['created_year'] = df2['created'].dt.year.astype('category').cat.codes
df2['created_month'] = df2['created'].dt.month.astype('category').cat.codes
df2['len_description'] = df2['description'].apply(lambda x: len(x.split(' ')))
df2['num_pics'] = df2['photos'].apply(len)

X = df2[new_feat].fillna(0)
y = vclf.predict_proba(X)

In [9]:
labels2idx = {label: i for i, label in enumerate(vclf.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [10]:
sub = pd.DataFrame()
sub['listing_id'] = df2['listing_id']
for l in ['high', 'medium', 'low']:
    sub[l] = y[:, labels2idx[l]]
sub.to_csv('submissionVoting.csv', index=False)

In [11]:
sub = pd.read_csv('submissionVoting.csv')
sub.head()

,listing_id,high,medium,low
0,7142618,0.234487,0.422990,0.342522
1,7210040,0.265117,0.346404,0.388479
2,7103890,0.145509,0.390160,0.464332
3,7143442,0.172184,0.376647,0.451169
4,6860601,0.132673,0.231785,0.635542
